### Invoke SparkContext

In [1]:
import findspark
findspark.init("C:/Users/Jonas/spark")
from pyspark import SparkContext, SparkConf
conf = SparkConf().setAppName('Spark Lab1')
sc = SparkContext(conf=conf)
sc

<SparkContext master=local[*] appName=Spark Lab1>

### SparkSession

In [2]:
from pyspark.sql import SparkSession

spark = SparkSession \
    .builder \
    .appName("Spark Project") \
    .config("spark.some.config.option", "some-value") \
    .getOrCreate()

In [3]:
spark.sparkContext

<SparkContext master=local[*] appName=Spark Lab1>

### Load data

In [4]:
from pyspark import SQLContext
reviews = spark.read.options(header=True).csv("amazon_reviews_us_Mobile_Electronics_v1_00.tsv", sep="\t")

### See structure of data

In [5]:
reviews.printSchema()

root
 |-- marketplace: string (nullable = true)
 |-- customer_id: string (nullable = true)
 |-- review_id: string (nullable = true)
 |-- product_id: string (nullable = true)
 |-- product_parent: string (nullable = true)
 |-- product_title: string (nullable = true)
 |-- product_category: string (nullable = true)
 |-- star_rating: string (nullable = true)
 |-- helpful_votes: string (nullable = true)
 |-- total_votes: string (nullable = true)
 |-- vine: string (nullable = true)
 |-- verified_purchase: string (nullable = true)
 |-- review_headline: string (nullable = true)
 |-- review_body: string (nullable = true)
 |-- review_date: string (nullable = true)



### Display data and show significant attributes

In [14]:
from pyspark.sql.functions import *

reviews.show()

reviews.groupBy('star_rating').count().orderBy(desc('count')).show()

reviews.groupBy('customer_id').count().orderBy(desc('count')).show()

#reviews.select('product_id').distinct().count()

+-----------+--------------+----------+--------------+--------------------+-----------+-----------+-----------------+--------------------+--------------------+
|customer_id|     review_id|product_id|product_parent|       product_title|star_rating|total_votes|verified_purchase|     review_headline|         review_body|
+-----------+--------------+----------+--------------+--------------------+-----------+-----------+-----------------+--------------------+--------------------+
|   20422322| R8MEA6IGAHO0B|B00MC4CED8|     217304173|BlackVue DR600GW-PMP|          5|          0|                Y|         Very Happy!|As advertised. Ev...|
|   40835037|R31LOQ8JGLPRLK|B00OQMFG1Q|     137313254|GENSSI GSM / GPS ...|          5|          1|                Y|           five star|          it's great|
|   51469641|R2Y0MM9YE6OP3P|B00QERR5CY|      82850235|iXCC Multi pack L...|          5|          0|                Y|        great cables|These work great ...|
|    4332923| RRB9C05HDOD4O|B00QUFTPV4| 

### See that there are more product id's than product titles

In [17]:
reviews.select('product_title').distinct().count()

24770

In [18]:
reviews.select('product_id').distinct().count()

25801

### Drop attributes of no importance

In [7]:
reviews = reviews.drop('vine', 'marketplace', 'helpful_votes', 'review_date', 'product_category')
reviews.show()

+-----------+--------------+----------+--------------+--------------------+-----------+-----------+-----------------+--------------------+--------------------+
|customer_id|     review_id|product_id|product_parent|       product_title|star_rating|total_votes|verified_purchase|     review_headline|         review_body|
+-----------+--------------+----------+--------------+--------------------+-----------+-----------+-----------------+--------------------+--------------------+
|   20422322| R8MEA6IGAHO0B|B00MC4CED8|     217304173|BlackVue DR600GW-PMP|          5|          0|                Y|         Very Happy!|As advertised. Ev...|
|   40835037|R31LOQ8JGLPRLK|B00OQMFG1Q|     137313254|GENSSI GSM / GPS ...|          5|          1|                Y|           five star|          it's great|
|   51469641|R2Y0MM9YE6OP3P|B00QERR5CY|      82850235|iXCC Multi pack L...|          5|          0|                Y|        great cables|These work great ...|
|    4332923| RRB9C05HDOD4O|B00QUFTPV4| 

### See if there are any missing values

In [8]:
reviews.select([count(when(col(column).isNull(), 1)).alias(column) for column in reviews.columns]).show()

+-----------+---------+----------+--------------+-------------+-----------+-----------+-----------------+---------------+-----------+
|customer_id|review_id|product_id|product_parent|product_title|star_rating|total_votes|verified_purchase|review_headline|review_body|
+-----------+---------+----------+--------------+-------------+-----------+-----------+-----------------+---------------+-----------+
|          0|        0|         0|             0|            0|          0|          0|                0|              0|          3|
+-----------+---------+----------+--------------+-------------+-----------+-----------+-----------------+---------------+-----------+



### See how many products a customer has reviewed as well as how many customers have reviewed a product

In [9]:
products_per_reviewer = reviews.select('customer_id', 'product_id').rdd.map(lambda line: (line[0], line[1])).groupByKey().mapValues(list)
products_per_reviewer.take(20)

reviewers_per_product = reviews.select('product_id', 'customer_id').rdd.map(lambda line: (line[0], line[1])).groupByKey().mapValues(list)
reviewers_per_product.first()

('B00LHXWPI4',
 ['10757416',
  '15932571',
  '17774962',
  '15120796',
  '1251653',
  '23091143',
  '17065307',
  '23742922',
  '34257199',
  '42238310',
  '17066831',
  '52670149',
  '12917544'])

### One-hot encoding 

In [34]:
from pyspark.ml.feature import StringIndexer, OneHotEncoder

# One-hot encode product_id.
indexer = StringIndexer(inputCol='product_id', outputCol='product_id_num')
prod_indx = indexer.fit(reviews).transform(reviews)
prod_indx.select(['product_id', 'product_id_num']).show(5)
encoder = OneHotEncoder(inputCol='product_id_num', outputCol='product_id_num_vec')
rev_enc = encoder.fit(prod_indx).transform(prod_indx)
rev_enc.select(['product_id', 'product_id_num_vec']).show(5)

# One-hot encode customer_id.
cust_indexer = StringIndexer(inputCol='customer_id', outputCol='customer_id_num')
cust_indx = cust_indexer.fit(rev_enc).transform(rev_enc)
cust_indx.select(['customer_id', 'customer_id_num']).show(5)
cust_encoder = OneHotEncoder(inputCol='customer_id_num', outputCol='customer_id_num_vec')
rev_enc = cust_encoder.fit(cust_indx).transform(cust_indx)
rev_enc.select(['customer_id', 'customer_id_num_vec']).show(5)

+----------+--------------+
|product_id|product_id_num|
+----------+--------------+
|B00MC4CED8|        3543.0|
|B00OQMFG1Q|       10502.0|
|B00QERR5CY|          13.0|
|B00QUFTPV4|        4651.0|
|B0067XVNTG|           6.0|
+----------+--------------+
only showing top 5 rows

+----------+--------------------+
|product_id|  product_id_num_vec|
+----------+--------------------+
|B00MC4CED8|(25800,[3543],[1.0])|
|B00OQMFG1Q|(25800,[10502],[1...|
|B00QERR5CY|  (25800,[13],[1.0])|
|B00QUFTPV4|(25800,[4651],[1.0])|
|B0067XVNTG|   (25800,[6],[1.0])|
+----------+--------------------+
only showing top 5 rows

+-----------+---------------+
|customer_id|customer_id_num|
+-----------+---------------+
|   20422322|        32552.0|
|   40835037|        68316.0|
|   51469641|        91970.0|
|    4332923|        73522.0|
|   44855305|          315.0|
+-----------+---------------+
only showing top 5 rows

+-----------+--------------------+
|customer_id| customer_id_num_vec|
+-----------+--------------